# Additional Queries

This notebook records some additional queries we made in addition to the simulation

## R2 v.s. r2

We are interested in comparing the correlation between exome and imputed data (r2) versus the imputation quality of imputed data (R2). Since correlation can only be computed between the overlapped variant between exome sequence data and imputed data, we generated another set of files for each gene for overlapped variants only.

### Extract variants

In [1]:
library(dplyr)
library(data.table)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last




In [2]:
df <- read.csv("~/project/imputation-rvtest/analysis/imputation_aggregated_analysis/hrc_topmed_exome/hrc_topmed_v3_exome_168206ids_rsq03_maf001_all_annot.csv.gz")

In [3]:
setwd("~/project/imputation-rvtest/analysis/imputation_aggregated_analysis/correlation/")

for(c in c(1,2)){
    for(maf in c(0.001, 0.005, 0.01)){
        if(maf == 0.001) {
            df1 <- df %>% filter(MAF_nfe_exome < 0.001 | is.na(MAF_nfe_exome))
        } else if (maf == 0.005) {
            df1 <- df %>% filter(MAF_nfe_exome >= 0.001 & MAF_nfe_exome < 0.005)
        } else {
            df1 <- df %>% filter(MAF_nfe_exome >= 0.005 & MAF_nfe_exome < 0.01)
        }
        
        topmed <- df1 %>% filter(Chr == c, R2_topmed != 0, R2_exome == 999)
        hrc <- df1 %>% filter(Chr == c, R2_hrc != 0, R2_exome == 999)
        
        print(sprintf("TOPMed: Chromosome %i, MAF %f, Number of variants = %i", c, maf, nrow(topmed)))
        print(sprintf("HRC: Chromosome %i, MAF %f, Number of variants = %i", c, maf, nrow(hrc)))
        
        counter = 0
        for(i in seq(1, nrow(topmed), 5000)){
            counter = counter + 1
            if(i+4999 > nrow(topmed)) end = nrow(topmed) else end = i+4999
            
            topmed[c(i:end),] %>% 
                select(ID_hg38) %>% 
                fwrite(sprintf("exome_topmed_ES_chr%i_maf%s_batch%i.txt", c, gsub('\\.', '', as.character(maf)), counter), col.names = FALSE)
            topmed[c(i:end),] %>% 
                select(ID_hg38) %>% 
                fwrite(sprintf("exome_topmed_TP_chr%i_maf%s_batch%i.txt", c, gsub('\\.', '', as.character(maf)), counter), col.names = FALSE)
        }
        
        counter = 0
        for(i in seq(1, nrow(hrc), 5000)){
            counter = counter + 1
            if(i+4999 > nrow(hrc)) end = nrow(hrc) else end = i+4999
                
            hrc[c(i:end),] %>% 
                select(ID_hg38) %>% 
                fwrite(sprintf("exome_hrc_ES_chr%i_maf%s_batch%i.txt", c, gsub('\\.', '', as.character(maf)), counter), col.names = FALSE)
            hrc[c(i:end),] %>% 
                select(ID_hg19) %>% 
                fwrite(sprintf("exome_hrc_HRC_chr%i_maf%s_batch%i.txt", c, gsub('\\.', '', as.character(maf)), counter), col.names = FALSE)
        }        
    }
}

[1] "TOPMed: Chromosome 1, MAF 0.001000, Number of variants = 108907"
[1] "HRC: Chromosome 1, MAF 0.001000, Number of variants = 15367"
[1] "TOPMed: Chromosome 1, MAF 0.005000, Number of variants = 3148"
[1] "HRC: Chromosome 1, MAF 0.005000, Number of variants = 3047"
[1] "TOPMed: Chromosome 1, MAF 0.010000, Number of variants = 807"
[1] "HRC: Chromosome 1, MAF 0.010000, Number of variants = 774"
[1] "TOPMed: Chromosome 2, MAF 0.001000, Number of variants = 76192"
[1] "HRC: Chromosome 2, MAF 0.001000, Number of variants = 10918"
[1] "TOPMed: Chromosome 2, MAF 0.005000, Number of variants = 2299"
[1] "HRC: Chromosome 2, MAF 0.005000, Number of variants = 2209"
[1] "TOPMed: Chromosome 2, MAF 0.010000, Number of variants = 544"
[1] "HRC: Chromosome 2, MAF 0.010000, Number of variants = 532"


In [4]:
cd ~/project/imputation-rvtest/analysis/imputation_aggregated_analysis/correlation

for chr in 1 2; do
    for maf in 0001 0005 001; do
        for dt in ES HRC; do
            extract_prefix='exome_hrc_'$dt'_chr'$chr'_maf'$maf'_batch'
            num_batch=$(ls $extract_prefix*'.txt'|wc -l)
            
            for ((i=1; i<=$num_batch; i++)); do
                script_name='/home/tl3031/project/imputation-rvtest/analysis/imputation_aggregated_analysis/correlation/script/'$extract_prefix$i'.sh'
                out_name='/home/tl3031/project/imputation-rvtest/analysis/imputation_aggregated_analysis/correlation/script/output/'$extract_prefix$i'.out'
                
                if [ "${dt}" = "ES" ]; then
                  bfile='/home/tl3031/project/imputation-rvtest/analysis/imputation_aggregated_analysis/exome/ukb23156_c'$chr'_maf001_LOF_missense_extracted'
                  plink_module='module load Plink/1.9.10'
                  plink_command='plink --bfile '$bfile' --extract '$extract_prefix$i'.txt --keep-allele-order --make-bed --export A --out '$extract_prefix$i
                else
                  bfile='/home/tl3031/project/imputation-rvtest/analysis/imputation_aggregated_analysis/hrc/hrc_chr'$chr'_merged_168206ids_rsq03_dose'
                  plink_module='module load Plink/2.00a'
                  plink_command='plink2 --bpfile '$bfile' --extract '$extract_prefix$i'.txt --make-bpgen --export A --out '$extract_prefix$i >> $script_name
                fi

                echo '#!/bin/bash' > $script_name
                echo '#SBATCH --job-name='$extract_prefix >> $script_name
                echo '#SBATCH --mem=10G' >> $script_name
                echo '#SBATCH --time=24:00:00' >> $script_name
                echo '#SBATCH --output=/home/tl3031/project/imputation-rvtest/analysis/imputation_aggregated_analysis/correlation/script/'$extract_prefix$i'_'%j'.out' >> $script_name
                echo '#SBATCH --error=/home/tl3031/project/imputation-rvtest/analysis/imputation_aggregated_analysis/correlation/script/'$extract_prefix$i'_'%j'.err' >> $script_name
                echo '#SBATCH -p CSG' >> $script_name
                echo '#SBATCH --mail-type=FAIL' >> $script_name
                echo '#SBATCH --mail-user tl3031@cumc.columbia.edu' >> $script_name
                echo 'source ~/mamba_activate.sh' >> $script_name
                echo $plink_module >> $script_name
                echo 'cd /home/tl3031/project/imputation-rvtest/analysis/imputation_aggregated_analysis/correlation' >> $script_name
                echo $plink_command >> $script_name
                echo 'echo "Number of variants in bim file:" >> '$out_name >> $script_name
                echo 'wc -l' $extract_prefix$i'.bim >> '$out_name >> $script_name
                echo 'echo "Number of variants in extract file:" >> '$out_name >> $script_name
                echo 'wc -l '$extract_prefix$i'.txt >> '$out_name >> $script_name
            done
        done
    done
done

In [6]:
cd ~/project/imputation-rvtest/analysis/imputation_aggregated_analysis/correlation

for chr in 1 2; do
    for maf in 0001 0005 001; do
        for dt in ES TP; do
            extract_prefix='exome_topmed_'$dt'_chr'$chr'_maf'$maf'_batch'
            num_batch=$(ls $extract_prefix*'.txt'|wc -l)
            
            for ((i=1; i<=$num_batch; i++)); do
                script_name='/home/tl3031/project/imputation-rvtest/analysis/imputation_aggregated_analysis/correlation/script/'$extract_prefix$i'.sh'
                out_name='/home/tl3031/project/imputation-rvtest/analysis/imputation_aggregated_analysis/correlation/script/output/'$extract_prefix$i'.out'
                
                if [ "${dt}" = "ES" ]; then
                  bfile='/home/tl3031/project/imputation-rvtest/analysis/imputation_aggregated_analysis/exome/ukb23156_c'$chr'_maf001_LOF_missense_extracted'
                  plink_module='module load Plink/1.9.10'
                  plink_command='plink --bfile '$bfile' --extract '$extract_prefix$i'.txt --keep-allele-order --make-bed --export A --out '$extract_prefix$i
                else
                  bfile='/home/tl3031/project/imputation-rvtest/analysis/imputation_aggregated_analysis/topmed_v3/topmed_chr'$chr'_merged_168206ids_rsq03_dose'
                  plink_module='module load Plink/2.00a'
                  plink_command='plink2 --bpfile '$bfile' --extract '$extract_prefix$i'.txt --make-bpgen --export A --out '$extract_prefix$i >> $script_name
                fi

                echo '#!/bin/bash' > $script_name
                echo '#SBATCH --job-name='$extract_prefix >> $script_name
                echo '#SBATCH --mem=10G' >> $script_name
                echo '#SBATCH --time=24:00:00' >> $script_name
                echo '#SBATCH --output=/home/tl3031/project/imputation-rvtest/analysis/imputation_aggregated_analysis/correlation/script/'$extract_prefix$i'_'%j'.out' >> $script_name
                echo '#SBATCH --error=/home/tl3031/project/imputation-rvtest/analysis/imputation_aggregated_analysis/correlation/script/'$extract_prefix$i'_'%j'.err' >> $script_name
                echo '#SBATCH -p CSG' >> $script_name
                echo '#SBATCH --mail-type=FAIL' >> $script_name
                echo '#SBATCH --mail-user tl3031@cumc.columbia.edu' >> $script_name
                echo 'source ~/mamba_activate.sh' >> $script_name
                echo $plink_module >> $script_name
                echo 'cd /home/tl3031/project/imputation-rvtest/analysis/imputation_aggregated_analysis/correlation' >> $script_name
                echo $plink_command >> $script_name
                echo 'echo "Number of variants in bim file:" >> '$out_name >> $script_name
                echo 'wc -l' $extract_prefix$i'.bim >> '$out_name >> $script_name
                echo 'echo "Number of variants in extract file:" >> '$out_name >> $script_name
                echo 'wc -l '$extract_prefix$i'.txt >> '$out_name >> $script_name
            done
        done
    done
done

### Calculate correlation

In [6]:
cd ~/project/imputation-rvtest/analysis/imputation_aggregated_analysis/correlation

In [7]:
script_dir="/mnt/vast/hpc/csg/tl3031/imputation-rvtest/analysis/imputation_aggregated_analysis/correlation/script/"
rm $script_dir*'corr'*'.sh'
rm $script_dir*'corr'*'.out'

for chr in 1 2; do
    for maf in 0001 0005 001; do
        for dt in HRC TP; do
            if [ "${dt}" = "HRC" ]; then
                prefix='exome_hrc'
            else
                prefix='exome_topmed'
            fi
            
            exome_prefix=$prefix'_ES_chr'$chr'_maf'$maf'_batch'
            imputed_prefix=$prefix'_'$dt'_chr'$chr'_maf'$maf'_batch'
            num_batch=$(ls $imputed_prefix*'.txt' | wc -l)
            
            for ((i=1; i<=$num_batch; i++)); do
                script_name=$script_dir$prefix'_chr'$chr'_maf'$maf'_corr_batch'$i'.sh'
                out_name=$script_dir'output/'$prefix'_chr'$chr'_maf'$maf'_corr_batch'$i'.out'
                command='Rscript '$script_dir'calc_corr.R '$exome_prefix$i' '$imputed_prefix$i' '$prefix'_chr'$chr'_maf'$maf'_corr_batch'$i'.csv '$dt
                echo '#!/bin/bash' >> $script_name
                echo '#SBATCH --job-name='$prefix'_chr'$chr'_maf'$maf'_corr_batch'$i >> $script_name
                echo '#SBATCH --mem=80G' >> $script_name
                echo '#SBATCH --time=24:00:00' >> $script_name
                echo '#SBATCH --output=/home/tl3031/project/imputation-rvtest/analysis/imputation_aggregated_analysis/correlation/script/'$prefix'_chr'$chr'_maf'$maf'_corr_batch'$i'_%j.out' >> $script_name
                echo '#SBATCH --error=/home/tl3031/project/imputation-rvtest/analysis/imputation_aggregated_analysis/correlation/script/'$prefix'_chr'$chr'_maf'$maf'_corr_batch'$i'_%j.err' >> $script_name
                echo '#SBATCH -p CSG' >> $script_name
                echo '#SBATCH --mail-type=FAIL' >> $script_name
                echo '#SBATCH --mail-user tl3031@cumc.columbia.edu' >> $script_name
                echo 'source ~/mamba_activate.sh' >> $script_name
                echo 'cd /home/tl3031/project/imputation-rvtest/analysis/imputation_aggregated_analysis/correlation' >> $script_name
                echo $command >> $script_name
                echo 'echo "Number of variants in bim file:" >> '$out_name >> $script_name
                echo 'wc -l' $exome_prefix$i'.bim >> '$out_name >> $script_name
                echo 'echo "Number of variants in correlation file:" >> '$out_name >> $script_name
                echo 'wc -l '$prefix'_chr'$chr'_maf'$maf'_corr_batch'$i'.csv >> '$out_name >> $script_name
            done
        done
    done
done

rm: cannot remove '/mnt/vast/hpc/csg/tl3031/imputation-rvtest/analysis/imputation_aggregated_analysis/correlation/script/*corr*.sh': No such file or directory
rm: cannot remove '/mnt/vast/hpc/csg/tl3031/imputation-rvtest/analysis/imputation_aggregated_analysis/correlation/script/*corr*.out': No such file or directory


## Check correlation and Rsq

In [1]:
library(dplyr)
library(data.table)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last




In [2]:
df_2 <- fread("~/project/imputation-rvtest/analysis/imputation_aggregated_analysis/hrc_topmed/hrc_topmed_v3_168206ids_rsq03_maf001_all_annot.csv.gz")
df_3 <- fread("~/project/imputation-rvtest/analysis/imputation_aggregated_analysis/hrc_topmed_exome/hrc_topmed_v3_exome_168206ids_rsq03_maf001_all_annot.csv.gz")

In [7]:
fname_lst <- list.files("/home/tl3031/project/imputation-rvtest/analysis/imputation_aggregated_analysis/correlation", pattern = ".csv", full.names = TRUE)

corr_df <- data.frame(matrix(ncol = 4, nrow = 0))
colnames(corr_df) <- c("snp_id", "corr", "rsq", "dataset")
for(i in fname_lst){
    dt <- case_when(
        grepl("hrc", i, fixed = TRUE) & grepl("maf001", i, fixed = TRUE) ~ "hrc_maf001",
        grepl("hrc", i, fixed = TRUE) & grepl("maf0005", i, fixed = TRUE) ~ "hrc_maf0005",
        grepl("hrc", i, fixed = TRUE) & grepl("maf0001", i, fixed = TRUE) ~ "hrc_maf0001",
        grepl("topmed", i, fixed = TRUE) & grepl("maf001", i, fixed = TRUE) ~ "topmed_maf001",
        grepl("topmed", i, fixed = TRUE) & grepl("maf0005", i, fixed = TRUE) ~ "topmed_maf0005",
        grepl("topmed", i, fixed = TRUE) & grepl("maf0001", i, fixed = TRUE) ~ "topmed_maf0001",
    )

    dataset_corr_df <- read.csv(i) %>% mutate(dataset = dt, corr = -corr)
    corr_df <- rbind(corr_df, dataset_corr_df)
}

dim(corr_df)

[1] 224744      4

In [9]:
hrc_corr_df <- corr_df %>% filter(grepl('hrc', dataset)) %>% rename("corr_hrc" = corr) %>% select(-rsq)
topmed_corr_df <- corr_df %>% filter(grepl('topmed', dataset)) %>% rename("corr_topmed" = corr) %>% select(-rsq)

In [10]:
hrc_topmed_df <- left_join(df_2, hrc_corr_df, by = c("ID_hg38" = "snp_id"))
hrc_topmed_df <- left_join(hrc_topmed_df, topmed_corr_df, by = c("ID_hg38" = "snp_id"))
hrc_topmed_df <- hrc_topmed_df %>% 
    filter(R2 != 0) %>%
    mutate(maf_cat = case_when(is.na(MAF_nfe_exome) | MAF_nfe_exome < 0.001 ~ "MAF < 0.001",
                               MAF_nfe_exome >= 0.001 & MAF_nfe_exome < 0.005 ~ "0.001 <= MAF < 0.005",
                               MAF_nfe_exome >= 0.005 & MAF_nfe_exome < 0.01 ~ "0.005 <= MAF < 0.01")) %>%
    group_by(maf_cat)

hrc_topmed_df <- hrc_topmed_df %>% 
    mutate(corr_hrc = ifelse(is.na(corr_hrc), 0, corr_hrc),
           corr_topmed = ifelse(is.na(corr_topmed), 0, corr_topmed))

In [11]:
hrc_topmed_exome_df <- left_join(df_3, hrc_corr_df, by = c("ID_hg38" = "snp_id"))
hrc_topmed_exome_df <- left_join(hrc_topmed_exome_df, topmed_corr_df, by = c("ID_hg38" = "snp_id"))
hrc_topmed_exome_df <- hrc_topmed_exome_df %>% 
    filter(R2 != 0) %>%
    mutate(maf_cat = case_when(is.na(MAF_nfe_exome) | MAF_nfe_exome < 0.001 ~ "MAF < 0.001",
                               MAF_nfe_exome >= 0.001 & MAF_nfe_exome < 0.005 ~ "0.001 <= MAF < 0.005",
                               MAF_nfe_exome >= 0.005 & MAF_nfe_exome < 0.01 ~ "0.005 <= MAF < 0.01")) %>%
    group_by(maf_cat)

hrc_topmed_exome_df <- hrc_topmed_exome_df %>% 
    mutate(corr_hrc = ifelse(is.na(corr_hrc), 0, corr_hrc),
           corr_topmed = ifelse(is.na(corr_topmed), 0, corr_topmed))

In [12]:
## hrc_R2

hrc_R2_summary <- df_2 %>% 
    filter(R2_hrc != 0) %>% 
    mutate(maf_cat = case_when(is.na(MAF_nfe_exome) | MAF_nfe_exome < 0.001 ~ "MAF < 0.001",
                               MAF_nfe_exome >= 0.001 & MAF_nfe_exome < 0.005 ~ "0.001 <= MAF < 0.005",
                               MAF_nfe_exome >= 0.005 & MAF_nfe_exome < 0.01 ~ "0.005 <= MAF < 0.01")) %>%
    group_by(maf_cat) %>%
    summarise(
        R2_mean = mean(R2_hrc),
        R2_std = sd(R2_hrc),
        R2_count = n()
    ) %>%
    mutate(maf_cat = factor(maf_cat, levels = c("MAF < 0.001", "0.001 <= MAF < 0.005", "0.005 <= MAF < 0.01")),
          data = "HRC", R2_se = R2_std/sqrt(R2_count)) %>% 
    arrange(maf_cat)

hrc_R2_summary 

maf_cat,R2_mean,R2_std,R2_count,data,R2_se
<fct>,<dbl>,<dbl>,<int>,<chr>,<dbl>
MAF < 0.001,0.6888802,0.2001369,28674,HRC,0.001181906
0.001 <= MAF < 0.005,0.8818392,0.1500554,5371,HRC,0.002047500
0.005 <= MAF < 0.01,0.9304061,0.1072783,1344,HRC,0.002926255


In [13]:
# topmed_R2

topmed_R2_summary <- df_2 %>% 
    filter(R2_topmed != 0) %>% 
    mutate(maf_cat = case_when(is.na(MAF_nfe_exome) | MAF_nfe_exome < 0.001 ~ "MAF < 0.001",
                               MAF_nfe_exome >= 0.001 & MAF_nfe_exome < 0.005 ~ "0.001 <= MAF < 0.005",
                               MAF_nfe_exome >= 0.005 & MAF_nfe_exome < 0.01 ~ "0.005 <= MAF < 0.01")) %>%
    group_by(maf_cat) %>%
    summarise(
        R2_mean = mean(R2_topmed),
        R2_std = sd(R2_topmed),
        R2_count = n()
    ) %>%
    mutate(maf_cat = factor(maf_cat, levels = c("MAF < 0.001", "0.001 <= MAF < 0.005", "0.005 <= MAF < 0.01")),
          data = "TOPMed", R2_se = R2_std/sqrt(R2_count)) %>% 
    arrange(maf_cat)

topmed_R2_summary

maf_cat,R2_mean,R2_std,R2_count,data,R2_se
<fct>,<dbl>,<dbl>,<int>,<chr>,<dbl>
MAF < 0.001,0.6672420,0.19658404,235264,TOPMed,0.0004052943
0.001 <= MAF < 0.005,0.9029821,0.11133815,5544,TOPMed,0.0014953140
0.005 <= MAF < 0.01,0.9496386,0.08539027,1387,TOPMed,0.0022928210


In [14]:
# hrc_r2

hrc_r2_summary <- hrc_topmed_exome_df  %>% 
    filter(R2_hrc !=0, R2_exome == 999) %>% 
    mutate(maf_cat = case_when(is.na(MAF_nfe_exome) | MAF_nfe_exome < 0.001 ~ "MAF < 0.001",
                               MAF_nfe_exome >= 0.001 & MAF_nfe_exome < 0.005 ~ "0.001 <= MAF < 0.005",
                               MAF_nfe_exome >= 0.005 & MAF_nfe_exome < 0.01 ~ "0.005 <= MAF < 0.01")) %>%
    group_by(maf_cat) %>%
    summarise(
        corr_mean = mean(corr_hrc),
        corr_std = sd(corr_hrc),
        corr_count = n()
    ) %>%
    mutate(maf_cat = factor(maf_cat, levels = c("MAF < 0.001", "0.001 <= MAF < 0.005", "0.005 <= MAF < 0.01")),
          data = "HRC", corr_se = corr_std/sqrt(corr_count)) %>% 
    arrange(maf_cat)

hrc_r2_summary

maf_cat,corr_mean,corr_std,corr_count,data,corr_se
<fct>,<dbl>,<dbl>,<int>,<chr>,<dbl>
MAF < 0.001,0.6694771,0.2801258,26285,HRC,0.001727823
0.001 <= MAF < 0.005,0.9053217,0.1474033,5256,HRC,0.002033198
0.005 <= MAF < 0.01,0.9379537,0.2003873,1306,HRC,0.005544963


In [15]:
# topmed_r2

topmed_r2_summary <- hrc_topmed_exome_df  %>% 
    filter(R2_topmed !=0, R2_exome == 999) %>% 
    mutate(maf_cat = case_when(is.na(MAF_nfe_exome) | MAF_nfe_exome < 0.001 ~ "MAF < 0.001",
                               MAF_nfe_exome >= 0.001 & MAF_nfe_exome < 0.005 ~ "0.001 <= MAF < 0.005",
                               MAF_nfe_exome >= 0.005 & MAF_nfe_exome < 0.01 ~ "0.005 <= MAF < 0.01")) %>%
    group_by(maf_cat) %>%
    summarise(
        corr_mean = mean(corr_topmed),
        corr_std = sd(corr_topmed),
        corr_count = n()
    ) %>%
    mutate(maf_cat = factor(maf_cat, levels = c("MAF < 0.001", "0.001 <= MAF < 0.005", "0.005 <= MAF < 0.01")),
          data = "TOPMed", corr_se = corr_std/sqrt(corr_count)) %>% 
    arrange(maf_cat)

topmed_r2_summary

maf_cat,corr_mean,corr_std,corr_count,data,corr_se
<fct>,<dbl>,<dbl>,<int>,<chr>,<dbl>
MAF < 0.001,0.6527062,0.3308834,185099,TOPMed,0.0007690826
0.001 <= MAF < 0.005,0.9357304,0.1101351,5447,TOPMed,0.0014922692
0.005 <= MAF < 0.01,0.9481036,0.2063360,1351,TOPMed,0.0056136745


In [16]:
## hrc_topmed_R2
hrc_topmed_df %>% 
    summarise(mean_R2 = mean(R2), se_R2 = sd(R2) / sqrt(n()), count_R2 = n()) %>% 
    mutate(maf_cat = factor(maf_cat, levels = c("MAF < 0.001", "0.001 <= MAF < 0.005", "0.005 <= MAF < 0.01"))) %>% 
    arrange(maf_cat)

maf_cat,mean_R2,se_R2,count_R2
<fct>,<dbl>,<dbl>,<int>
MAF < 0.001,0.6730641,0.0004047096,238500
0.001 <= MAF < 0.005,0.9206836,0.0015047751,5725
0.005 <= MAF < 0.01,0.9463442,0.0025661029,1438


In [17]:
## hrc_topmed_r2

hrc_topmed_df  %>% 
    mutate(corr = ifelse(source == "hrc", corr_hrc, corr_topmed)) %>%
    filter(corr != 0) %>% 
    summarise(mean_corr = mean(corr), se_corr = sd(corr) / sqrt(n()), count_corr = n()) %>% 
    mutate(maf_cat = factor(maf_cat, levels = c("MAF < 0.001", "0.001 <= MAF < 0.005", "0.005 <= MAF < 0.01"))) %>% 
    arrange(maf_cat)

maf_cat,mean_corr,se_corr,count_corr
<fct>,<dbl>,<dbl>,<int>
MAF < 0.001,0.6475809,0.0007692251,187355
0.001 <= MAF < 0.005,0.9370379,0.0018199634,5576
0.005 <= MAF < 0.01,0.9431791,0.0056898435,1387


### T-test for R2 vs r2

#### Table S2

In [18]:
# hrc_corr_df <- corr_df %>% 
#     mutate(maf_cat = case_when(grepl("0001", dataset) ~ "MAF < 0.001",
#                                grepl("0005", dataset) ~ "0.001 <= MAF < 0.005",
#                                grepl("001", dataset)  ~ "0.005 <= MAF < 0.01",
#                                TRUE ~ "dataset")) %>%
#     filter(grepl("hrc_", dataset)) %>%
#     select(snp_id, corr, maf_cat)

# topmed_corr_df <- corr_df %>% 
#     mutate(maf_cat = case_when(grepl("0001", dataset) ~ "MAF < 0.001",
#                                grepl("0005", dataset) ~ "0.001 <= MAF < 0.005",
#                                grepl("001", dataset)  ~ "0.005 <= MAF < 0.01",
#                                TRUE ~ "dataset")) %>%
#     filter(grepl("topmed_", dataset)) %>%
#     select(snp_id, corr, maf_cat)

In [19]:
hrc_r2_df <- df_3 %>% 
    filter(R2_hrc != 0) %>% 
    mutate(maf_cat = case_when(is.na(MAF_nfe_exome) | MAF_nfe_exome < 0.001 ~ "MAF < 0.001",
                               MAF_nfe_exome >= 0.001 & MAF_nfe_exome < 0.005 ~ "0.001 <= MAF < 0.005",
                               MAF_nfe_exome >= 0.005 & MAF_nfe_exome < 0.01 ~ "0.005 <= MAF < 0.01")) %>%
    select(ID_hg38, R2_hrc, maf_cat)

topmed_r2_df <- df_3 %>% 
    filter(R2_topmed != 0) %>% 
    mutate(maf_cat = case_when(is.na(MAF_nfe_exome) | MAF_nfe_exome < 0.001 ~ "MAF < 0.001",
                               MAF_nfe_exome >= 0.001 & MAF_nfe_exome < 0.005 ~ "0.001 <= MAF < 0.005",
                               MAF_nfe_exome >= 0.005 & MAF_nfe_exome < 0.01 ~ "0.005 <= MAF < 0.01")) %>%
    select(ID_hg38, R2_topmed, maf_cat)

In [20]:
## check if the R2 is statistically different for HRC v.s. TOPMed

r2_df <- data.frame(matrix(ncol=9, nrow = 0))
colnames(r2_df) <- c("r2_mean_hrc", "r2_se_hrc", "r2_n_hrc", "r2_mean_topmed", "r2_se_topmed", "r2_n_topmed", "test_stat", "p_value", "maf_cat")

for(maf in c("MAF < 0.001", "0.001 <= MAF < 0.005", "0.005 <= MAF < 0.01")){
    hrc <- hrc_r2_df %>% filter(maf_cat == maf)
    topmed <- topmed_r2_df %>% filter(maf_cat == maf)
    
    test_result <- t.test(hrc$R2_hrc, topmed$R2_topmed)
    
    sub_r2_df <- data.frame("r2_mean_hrc" = mean(hrc$R2_hrc),
                              "r2_se_hrc" = sd(hrc$R2_hrc) / sqrt(length(hrc$R2_hrc)),
                              "r2_n_hrc" = length(hrc$R2_hrc), 
                              "r2_mean_topmed" = mean(topmed$R2_topmed),
                              "r2_se_topmed" = sd(topmed$R2_topmed) / sqrt(length(topmed$R2_topmed)),
                              "r2_n_topmed" = length(topmed$R2_topmed), 
                              "test_stat" = test_result$statistic, 
                              "p_value" = test_result$p.value,
                              "maf_cat" = maf)
    r2_df = rbind(r2_df, sub_r2_df)
}

r2_df

,r2_mean_hrc,r2_se_hrc,r2_n_hrc,r2_mean_topmed,r2_se_topmed,r2_n_topmed,test_stat,p_value,maf_cat
,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<chr>
t,0.6888802,0.001181906,28674,0.6672420,0.0004052943,235264,17.317953,6.463932e-67,MAF < 0.001
t1,0.8818392,0.002047500,5371,0.9029821,0.0014953140,5544,-8.339087,8.483926e-17,0.001 <= MAF < 0.005
t2,0.9304061,0.002926255,1344,0.9496386,0.0022928210,1387,-5.173460,2.475786e-07,0.005 <= MAF < 0.01


In [21]:
## check if the RSQ is statistically different for HRC v.s. HRC_TOPMed

r2_df <- data.frame(matrix(ncol=9, nrow = 0))
colnames(r2_df) <- c("r2_mean_hrc", "r2_se_hrc", "r2_n_hrc", "r2_mean_hrc_topmed", "r2_se_hrc_topmed", "r2_n_hrc_topmed", "test_stat", "p_value", "maf_cat")

for(maf in c("MAF < 0.001", "0.001 <= MAF < 0.005", "0.005 <= MAF < 0.01")){
    hrc <- hrc_r2_df %>% filter(maf_cat == maf)
    hrc_topmed <- hrc_topmed_df %>% filter(maf_cat == maf)
    
    test_result <- t.test(hrc$R2_hrc, hrc_topmed$R2)
    
    sub_r2_df <- data.frame("r2_mean_hrc" = mean(hrc$R2_hrc),
                              "r2_se_hrc" = sd(hrc$R2_hrc) / sqrt(length(hrc$R2_hrc)),
                              "r2_n_hrc" = length(hrc$R2_hrc), 
                              "r2_mean_hrc_topmed" = mean(hrc_topmed$R2),
                              "r2_se_hrc_topmed" = sd(hrc_topmed$R2) / sqrt(length(hrc_topmed$R2)),
                              "r2_n_hrc_topmed" = length(hrc_topmed$R2), 
                              "test_stat" = test_result$statistic, 
                              "p_value" = test_result$p.value,
                              "maf_cat" = maf)
    r2_df = rbind(r2_df, sub_r2_df)
}

r2_df

,r2_mean_hrc,r2_se_hrc,r2_n_hrc,r2_mean_hrc_topmed,r2_se_hrc_topmed,r2_n_hrc_topmed,test_stat,p_value,maf_cat
,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<chr>
t,0.6888802,0.001181906,28674,0.6730641,0.0004047096,238500,12.660195,1.178015e-36,MAF < 0.001
t1,0.8818392,0.002047500,5371,0.9206836,0.0015047751,5725,-15.287131,3.615025e-52,0.001 <= MAF < 0.005
t2,0.9304061,0.002926255,1344,0.9463442,0.0025661029,1438,-4.095056,4.344396e-05,0.005 <= MAF < 0.01


In [22]:
## check if the RSQ is statistically different for TOPMed v.s. HRC_TOPMed

r2_df <- data.frame(matrix(ncol=9, nrow = 0))
colnames(r2_df) <- c("r2_mean_topmed", "r2_se_topmed", "r2_n_topmed", "r2_mean_hrc_topmed", "r2_se_hrc_topmed", "r2_n_hrc_topmed", "test_stat", "p_value", "maf_cat")

for(maf in c("MAF < 0.001", "0.001 <= MAF < 0.005", "0.005 <= MAF < 0.01")){
    topmed <- topmed_r2_df %>% filter(maf_cat == maf)
    hrc_topmed <- hrc_topmed_df %>% filter(maf_cat == maf)
    
    test_result <- t.test(hrc$R2_hrc, hrc_topmed$R2)
    
    sub_r2_df <- data.frame("r2_mean_topmed" = mean(topmed$R2_topmed),
                              "r2_se_topmed" = sd(topmed$R2_topmed) / sqrt(length(topmed$R2_topmed)),
                              "r2_n_topmed" = length(topmed$R2_topmed), 
                              "r2_mean_hrc_topmed" = mean(hrc_topmed$R2),
                              "r2_se_hrc_topmed" = sd(hrc_topmed$R2) / sqrt(length(hrc_topmed$R2)),
                              "r2_n_hrc_topmed" = length(hrc_topmed$R2), 
                              "test_stat" = test_result$statistic, 
                              "p_value" = test_result$p.value,
                              "maf_cat" = maf)
    r2_df = rbind(r2_df, sub_r2_df)
}

r2_df

,r2_mean_topmed,r2_se_topmed,r2_n_topmed,r2_mean_hrc_topmed,r2_se_hrc_topmed,r2_n_hrc_topmed,test_stat,p_value,maf_cat
,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<chr>
t,0.6672420,0.0004052943,235264,0.6730641,0.0004047096,238500,87.113222,0.000000e+00,MAF < 0.001
t1,0.9029821,0.0014953140,5544,0.9206836,0.0015047751,5725,2.954738,3.163951e-03,0.001 <= MAF < 0.005
t2,0.9496386,0.0022928210,1387,0.9463442,0.0025661029,1438,-4.095056,4.344396e-05,0.005 <= MAF < 0.01


#### Table S3

In [23]:
## check if the correlation and R2 is statistically different from each other 

corr_rsq_df <- data.frame(matrix(ncol=10, nrow = 0))
colnames(corr_rsq_df) <- c("dataset", "maf_cat", "corr_mean", "corr_se", "corr_n", "rsq_mean", "rsq_se", "rsq_n", "test_stat", "p_value")

for(maf in c("MAF < 0.001", "0.001 <= MAF < 0.005", "0.005 <= MAF < 0.01")){
    hrc_corr <- hrc_topmed_exome_df %>% filter(R2_hrc !=0, R2_exome == 999) %>% filter(maf_cat == maf) %>% pull(corr_hrc)
    hrc_rsq <- hrc_r2_df %>% filter(R2_hrc !=0) %>% filter(maf_cat == maf) %>% pull(R2_hrc)
    
    topmed_corr <- hrc_topmed_exome_df %>% filter(R2_topmed !=0, R2_exome == 999) %>% filter(maf_cat == maf) %>% pull(corr_topmed)
    topmed_rsq <- hrc_topmed_exome_df %>% filter(R2_topmed !=0) %>% filter(maf_cat == maf) %>% pull(R2_topmed)
    
    hrc_test_result <- t.test(hrc_corr, hrc_rsq)
    topmed_test_result <- t.test(topmed_corr, topmed_rsq)
    
    sub_df_hrc <- data.frame("dataset" = "HRC", 
                             "maf_cat" = maf, 
                             "corr_mean" = mean(hrc_corr), 
                             "corr_se" = sd(hrc_corr) / sqrt(length(hrc_corr)),
                             "corr_n" = length(hrc_corr), 
                             "rsq_mean" = mean(hrc_rsq), 
                             "rsq_se" = sd(hrc_rsq) / sqrt(length(hrc_rsq)),
                             "rsq_n" = length(hrc_rsq), 
                             "test_stat" = hrc_test_result$statistic, 
                             "p_value" = hrc_test_result$p.value)
    
    sub_df_topmed <- data.frame("dataset" = "TOPMed", 
                                "maf_cat" = maf, 
                                "corr_mean" = mean(topmed_corr), 
                                "corr_se" = sd(topmed_corr) / sqrt(length(topmed_corr)),
                                "corr_n" = length(topmed_corr), 
                                "rsq_mean" = mean(topmed_rsq), 
                                "rsq_se" = sd(topmed_rsq) / sqrt(length(topmed_rsq)),
                                "rsq_n" = length(topmed_rsq), 
                                "test_stat" = topmed_test_result$statistic, 
                                "p_value" = topmed_test_result$p.value)
    
    corr_rsq_df = rbind(corr_rsq_df, sub_df_hrc, sub_df_topmed)
}

corr_rsq_df %>% arrange(dataset)

,dataset,maf_cat,corr_mean,corr_se,corr_n,rsq_mean,rsq_se,rsq_n,test_stat,p_value
,<chr>,<chr>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>
t,HRC,MAF < 0.001,0.6694771,0.0017278228,26285,0.6888802,0.0011819063,28674,-9.2687780,1.959701e-20
t2,HRC,0.001 <= MAF < 0.005,0.9053217,0.0020331977,5256,0.8818392,0.0020475001,5371,8.1380721,4.464713e-16
t4,HRC,0.005 <= MAF < 0.01,0.9379537,0.0055449631,1306,0.9304061,0.0029262554,1344,1.2038127,2.288056e-01
t1,TOPMed,MAF < 0.001,0.6527062,0.0007690826,185099,0.6672420,0.0004052943,235264,-16.7205181,9.954842e-63
t3,TOPMed,0.001 <= MAF < 0.005,0.9357304,0.0014922692,5447,0.9029821,0.0014953140,5544,15.5018759,1.243385e-53
t5,TOPMed,0.005 <= MAF < 0.01,0.9481036,0.0056136745,1351,0.9496386,0.0022928210,1387,-0.2531366,8.001918e-01


In [24]:
hrc_topmed_full_df <- hrc_topmed_exome_df %>% 
    filter(R2 != 0) %>%
    mutate(maf_cat = case_when(is.na(MAF_nfe_exome) | MAF_nfe_exome < 0.001 ~ "MAF < 0.001",
                               MAF_nfe_exome >= 0.001 & MAF_nfe_exome < 0.005 ~ "0.001 <= MAF < 0.005",
                               MAF_nfe_exome >= 0.005 & MAF_nfe_exome < 0.01 ~ "0.005 <= MAF < 0.01"),
          corr = ifelse(R2 == R2_hrc, corr_hrc, corr_topmed))

corr_rsq_hrc_topmed_df <- data.frame(matrix(ncol=10, nrow = 0))
colnames(corr_rsq_hrc_topmed_df) <- c("dataset", "maf_cat", "corr_mean", "corr_se", "corr_n", "rsq_mean", "rsq_se", "rsq_n", "test_stat", "p_value")

for(maf in c("MAF < 0.001", "0.001 <= MAF < 0.005", "0.005 <= MAF < 0.01")){
    subdf <- hrc_topmed_full_df %>% filter(maf_cat == maf)
    corr <- subdf %>% filter(R2_exome==999, R2 != 0) %>% pull(corr)
    test_result <- t.test(corr, subdf$R2)
    
    sub_df <- data.frame("dataset" = "HRC_TOPMed", 
                             "maf_cat" = maf, 
                             "corr_mean" = mean(corr), 
                             "corr_se" = sd(corr) / sqrt(length(corr)),
                             "corr_n" = length(corr), 
                             "rsq_mean" = mean(subdf$R2), 
                             "rsq_se" = sd(subdf$R2) / sqrt(length(subdf$R2)),
                             "rsq_n" = length(subdf$R2), 
                             "test_stat" = test_result$statistic, 
                             "p_value" = test_result$p.value)
    
    corr_rsq_hrc_topmed_df = rbind(sub_df, corr_rsq_hrc_topmed_df)
}
corr_rsq_hrc_topmed_df %>% arrange(dataset)

,dataset,maf_cat,corr_mean,corr_se,corr_n,rsq_mean,rsq_se,rsq_n,test_stat,p_value
,<chr>,<chr>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>
t,HRC_TOPMed,0.005 <= MAF < 0.01,0.9431791,0.0056898435,1387,0.9463442,0.0025661029,1438,-0.5070834,6.121542e-01
t2,HRC_TOPMed,0.001 <= MAF < 0.005,0.9370379,0.0018199634,5576,0.9206836,0.0015047751,5725,6.9254575,4.591322e-12
t1,HRC_TOPMed,MAF < 0.001,0.6475809,0.0007692251,187355,0.6730641,0.0004047096,238500,-29.3182964,1.151375e-188


#### Overlapped variants - Table S3

In [25]:
df_overlap <- df_3 %>% 
    filter(R2_topmed != 0, R2_hrc != 0) %>% 
    left_join(corr_df %>% filter(grepl("hrc", dataset)) %>% rename("corr_hrc" = "corr") %>% select(-dataset, -rsq), by = c("ID_hg38" = "snp_id")) %>%
    left_join(corr_df %>% filter(grepl("topmed", dataset)) %>% rename("corr_topmed" = "corr") %>% select(-dataset, -rsq), by = c("ID_hg38" = "snp_id")) %>%
    mutate(corr_hrc = ifelse(is.na(corr_hrc), 0, corr_hrc),
           corr_topmed = ifelse(is.na(corr_topmed), 0, corr_topmed)) %>%
    mutate(corr = ifelse(source == "hrc", corr_hrc, corr_topmed)) %>%
    mutate(maf_cat = case_when(is.na(MAF_nfe_exome) | MAF_nfe_exome < 0.001 ~ "MAF < 0.001",
                               MAF_nfe_exome >= 0.001 & MAF_nfe_exome < 0.005 ~ "0.001 <= MAF < 0.005",
                               MAF_nfe_exome >= 0.005 & MAF_nfe_exome < 0.01 ~ "0.005 <= MAF < 0.01"))

In [26]:
corr_rsq_overlap_df <- data.frame(matrix(ncol=10, nrow = 0))
colnames(corr_rsq_overlap_df) <- c("dataset", "maf_cat", "corr_mean", "corr_se", "corr_n", "rsq_mean", "rsq_se", "rsq_n", "test_stat", "p_value")

for(maf in c("MAF < 0.001", "0.001 <= MAF < 0.005", "0.005 <= MAF < 0.01")){
    df_overlap_maf <- df_overlap %>% filter(maf_cat == maf) %>% filter(!is.na(corr_hrc) & !is.na(corr_topmed))
    hrc_corr <- df_overlap_maf %>% filter(corr_hrc != 0) %>% pull(corr_hrc)
    hrc_rsq <- df_overlap_maf %>% pull(R2_hrc)
    
    topmed_corr <- df_overlap_maf %>% filter(corr_topmed != 0) %>% pull(corr_topmed)
    topmed_rsq <- df_overlap_maf %>% pull(R2_topmed)
    
    hrc_test_result <- t.test(hrc_corr, hrc_rsq)
    topmed_test_result <- t.test(topmed_corr, topmed_rsq)
    
    sub_df_hrc <- data.frame("dataset" = "HRC", 
                             "maf_cat" = maf, 
                             "corr_mean" = mean(hrc_corr), 
                             "corr_se" = sd(hrc_corr) / sqrt(length(hrc_corr)),
                             "corr_n" = length(hrc_corr), 
                             "rsq_mean" = mean(hrc_rsq), 
                             "rsq_se" = sd(hrc_rsq) / sqrt(length(hrc_rsq)),
                             "rsq_n" = length(hrc_rsq), 
                             "test_stat" = hrc_test_result$statistic, 
                             "p_value" = hrc_test_result$p.value)
    
    sub_df_topmed <- data.frame("dataset" = "TOPMed", 
                                "maf_cat" = maf, 
                                "corr_mean" = mean(topmed_corr), 
                                "corr_se" = sd(topmed_corr) / sqrt(length(topmed_corr)),
                                "corr_n" = length(topmed_corr), 
                                "rsq_mean" = mean(topmed_rsq), 
                                "rsq_se" = sd(topmed_rsq) / sqrt(length(topmed_rsq)),
                                "rsq_n" = length(topmed_rsq), 
                                "test_stat" = topmed_test_result$statistic, 
                                "p_value" = topmed_test_result$p.value)
    
    corr_rsq_overlap_df = rbind(corr_rsq_overlap_df, sub_df_hrc, sub_df_topmed)
}

corr_rsq_overlap_df %>% arrange(dataset)

,dataset,maf_cat,corr_mean,corr_se,corr_n,rsq_mean,rsq_se,rsq_n,test_stat,p_value
,<chr>,<chr>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>
t,HRC,MAF < 0.001,0.6895686,0.001702326,24029,0.7058350,0.001224586,25438,-7.7569040,8.888825e-15
t2,HRC,0.001 <= MAF < 0.005,0.9125491,0.001746531,5127,0.8890460,0.001964387,5190,8.9415020,4.504836e-19
t4,HRC,0.005 <= MAF < 0.01,0.9426791,0.005497617,1270,0.9392462,0.002486867,1293,0.5689251,5.694792e-01
t1,TOPMed,MAF < 0.001,0.8327140,0.001125612,24029,0.7340780,0.001146041,25438,61.4031828,0.000000e+00
t3,TOPMed,0.001 <= MAF < 0.005,0.9452535,0.001086329,5127,0.9102365,0.001406263,5190,19.7058214,8.557059e-85
t5,TOPMed,0.005 <= MAF < 0.01,0.9594623,0.005349063,1270,0.9582250,0.001912479,1293,0.2178049,8.276091e-01


#### Overlapped variants - Table S2

In [27]:
rsq_overlap_df <- data.frame(matrix(ncol=10, nrow = 0))
colnames(rsq_overlap_df) <- c("maf_cat", "rsq_mean", "rsq_se", "rsq_n", "test_stat", "p_value")

for(maf in c("MAF < 0.001", "0.001 <= MAF < 0.005", "0.005 <= MAF < 0.01")){
    hrc_rsq <- df_overlap %>% filter(maf_cat == maf) %>% pull(R2_hrc)
    topmed_rsq <- df_overlap %>% filter(maf_cat == maf) %>% pull(R2_topmed)
    
    test_result <- t.test(hrc_rsq, topmed_rsq)
    t_statistic <- (mean(hrc_rsq) - mean(topmed_rsq)) / sqrt((var(hrc_rsq)/length(hrc_rsq)) + (var(topmed_rsq)/length(topmed_rsq)))
    
    t.value = unname(test_result$statistic)
    df = unname(test_result$parameter[1])
    p = as.character(2*pt(-abs(t.value), df))
    # print(.N(test_result$p.value))
    
    sub_df <- data.frame("maf_cat" = maf, 
                         "rsq_hrc_mean" = mean(hrc_rsq), 
                         "rsq_hrc_se" = sd(hrc_rsq) / sqrt(length(hrc_rsq)),
                         # "rsq_hrc_se" = sd(hrc_rsq),
                         "rsq_hrc_n" = length(hrc_rsq), 
                         "rsq_topmed_mean" = mean(topmed_rsq), 
                         "rsq_topmed_se" = sd(topmed_rsq) / sqrt(length(topmed_rsq)),
                         # "rsq_topmed_se" = sd(topmed_rsq),
                         "rsq_topmed_n" = length(topmed_rsq), 
                         "test_stat" = test_result$statistic, 
                         "p_value" = test_result$p.value)
    
    rsq_overlap_df = rbind(rsq_overlap_df, sub_df)
}

rsq_overlap_df

,maf_cat,rsq_hrc_mean,rsq_hrc_se,rsq_hrc_n,rsq_topmed_mean,rsq_topmed_se,rsq_topmed_n,test_stat,p_value
,<chr>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>
t,MAF < 0.001,0.7058350,0.001224586,25438,0.7340780,0.001146041,25438,-16.839329,1.871105e-63
t1,0.001 <= MAF < 0.005,0.8890460,0.001964387,5190,0.9102365,0.001406263,5190,-8.771380,2.072379e-18
t2,0.005 <= MAF < 0.01,0.9392462,0.002486867,1293,0.9582250,0.001912479,1293,-6.049575,1.677485e-09


In [28]:
## check if the RSQ is statistically different v.s. HRC_TOPMed

r2_df <- data.frame(matrix(ncol=10, nrow = 0))
colnames(r2_df) <- c("data", "r2_mean", "r2_se", "r2_n", "r2_mean_hrc_topmed", "r2_se_hrc_topmed", "r2_n_hrc_topmed", "test_stat", "p_value", "maf_cat")

for(maf in c("MAF < 0.001", "0.001 <= MAF < 0.005", "0.005 <= MAF < 0.01")){
    hrc_r2 <- df_overlap %>% filter(maf_cat == maf) %>% pull(R2_hrc)
    topmed_r2 <- df_overlap %>% filter(maf_cat == maf) %>% pull(R2_topmed)
    hrc_topmed <- hrc_topmed_df %>% filter(maf_cat == maf) %>% pull(R2)
    
    hrc_test_result <- t.test(hrc_r2, hrc_topmed)
    topmed_test_result <- t.test(topmed_r2, hrc_topmed)
    
    hrc_t_statistic <- (mean(hrc_r2) - mean(hrc_topmed)) / sqrt((var(hrc_r2)/length(hrc_r2)) + (var(hrc_topmed)/length(hrc_topmed)))
    topmed_t_statistic <- (mean(topmed_r2) - mean(hrc_topmed)) / sqrt((var(topmed_r2)/length(topmed_r2)) + (var(hrc_topmed)/length(hrc_topmed)))


    hrc_r2_df <- data.frame("data" = "HRC", 
                            "r2_mean" = mean(hrc_r2),
                            "r2_se" = sd(hrc_r2) / sqrt(length(hrc_r2)),
                            "r2_n" = length(hrc_r2), 
                            "r2_mean_hrc_topmed" = mean(hrc_topmed),
                            "r2_se_hrc_topmed" = sd(hrc_topmed) / sqrt(length(hrc_topmed)),
                            "r2_n_hrc_topmed" = length(hrc_topmed), 
                            "test_stat" = hrc_test_result$statistic, 
                            "p_value" = hrc_test_result$p.value,
                            "maf_cat" = maf)
    
    
    topmed_r2_df <- data.frame("data" = "TOPMed",
                               "r2_mean" = mean(topmed_r2),
                               "r2_se" = sd(topmed_r2) / sqrt(length(topmed_r2)),
                               "r2_n" = length(topmed_r2), 
                               "r2_mean_hrc_topmed" = mean(hrc_topmed),
                               "r2_se_hrc_topmed" = sd(hrc_topmed) / sqrt(length(hrc_topmed)),
                               "r2_n_hrc_topmed" = length(hrc_topmed), 
                               "test_stat" = topmed_test_result$statistic, 
                               "p_value" = topmed_test_result$p.value,
                                "maf_cat" = maf)
    
    r2_df = rbind(r2_df, hrc_r2_df, topmed_r2_df)
}

r2_df

,data,r2_mean,r2_se,r2_n,r2_mean_hrc_topmed,r2_se_hrc_topmed,r2_n_hrc_topmed,test_stat,p_value,maf_cat
,<chr>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<chr>
t,HRC,0.7058350,0.001224586,25438,0.6730641,0.0004047096,238500,25.409129,5.414201e-141,MAF < 0.001
t1,TOPMed,0.7340780,0.001146041,25438,0.6730641,0.0004047096,238500,50.200632,0.000000e+00,MAF < 0.001
t2,HRC,0.8890460,0.001964387,5190,0.9206836,0.0015047751,5725,-12.785414,3.872412e-37,0.001 <= MAF < 0.005
t3,TOPMed,0.9102365,0.001406263,5190,0.9206836,0.0015047751,5725,-5.072410,3.992675e-07,0.001 <= MAF < 0.005
t4,HRC,0.9392462,0.002486867,1293,0.9463442,0.0025661029,1438,-1.986312,4.709863e-02,0.005 <= MAF < 0.01
t5,TOPMed,0.9582250,0.001912479,1293,0.9463442,0.0025661029,1438,3.712317,2.097021e-04,0.005 <= MAF < 0.01


In [58]:
.Machine$double.xmin

[1] 2.225074e-308

## Power for overlapped variants of TOPMed and Exome

We are going to perform the simulation with an overlapped set of variants from Exome and TOPMed using the already simulated phenotype from previous simulations. 

**Parameters**: all variants causal, OR = 1.5, disease prevalence = 0.1, $R^2$ = 0.3

### Create per gene genotype file

In [1]:
library(dplyr)
library(data.table)

annot <- fread("~/project/imputation-rvtest/analysis/imputation_aggregated_analysis/hrc_topmed_exome/hrc_topmed_v3_exome_168206ids_rsq03_maf001_annot.csv.gz")


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last




In [2]:
setwd("~/project/imputation-rvtest/workflows/imputation_aggregated_analysis/scripts")

In [3]:
for(maf in c(0.01, 0.005, 0.001)){
    annot_overlap <- annot %>% filter(MAF_nfe_exome < maf, !is.na(R2_topmed), R2_topmed != 0, R2_exome == 999)
    print(length(unique(annot_overlap$Gene.refGene)))

    maf_c <- gsub("\\.", "", as.character(maf))
    maf_c <- paste0("maf", maf_c)
    command_fname <- sprintf("./extract_overlapped_topmed_%s.sh", maf_c)
    
    if(file.exists(command_fname)) unlink(command_fname)
    file.create(command_fname)
    fileConn <- file(command_fname, "w")
    header_lines <- c(
        "#!/bin/bash",
        "#SBATCH --mem=60G",
        "#SBATCH --time=24:00:00",
        sprintf("#SBATCH --job-name=extract_overlapped_topmed_%s", maf_c),
        sprintf("#SBATCH --output=~/project/imputation-rvtest/workflows/imputation_aggregated_analysis/scripts/extract_overlapped_topmed_%s.out", maf_c),
        sprintf("#SBATCH --error=~/project/imputation-rvtest/workflows/imputation_aggregated_analysis/scripts/extract_overlapped_topmed_%s.err", maf_c),
        "#SBATCH -p CSG",
        "#SBATCH --mail-type=FAIL",
        "#SBATCH --mail-user=tl3031@cumc.columbia.edu",
        "source ~/mamba_activate.sh",
        sprintf("cd ~/project/imputation-rvtest/workflows/imputation_aggregated_analysis/gene/exome_topmed_overlap/topmed_%s", maf_c),
        "module load Plink/2.00a"
    )
    writeLines(header_lines, fileConn)
    close(fileConn)

    for(g in unique(annot_overlap$Gene.refGene)){
        annot_gene <- annot_overlap %>% 
            filter(is.na(MAF_nfe_exome) | MAF_nfe_exome < maf) %>% 
            filter(Gene.refGene == g) 
        
        annot_gene %>% 
            select(ID_hg38) %>% 
            fwrite(sprintf("~/project/imputation-rvtest/workflows/imputation_aggregated_analysis/gene/exome_topmed_overlap/topmed_%s/%s", maf_c, g))
        
        chr <- annot_gene[1,]$Chr
        
        command <- sprintf("plink2 --bpfile ~/project/imputation-rvtest/analysis/imputation_aggregated_analysis/topmed_v3/topmed_chr%i_rsq03_%s_LOF_missense_extracted \\
                            --extract %s --export A --make-bpgen --out %s;", chr, maf_c, g, g)
        fileConn <- file(command_fname, "a")
        writeLines(command, fileConn)
        close(fileConn)
    }
}

[1] 3097
[1] 3097
[1] 3097


In [7]:
for(maf in c(0.01, 0.005, 0.001)){
    annot_overlap <- annot %>% filter(MAF_nfe_exome < maf, !is.na(R2_topmed), R2_topmed != 0, R2_exome == 999)
    print(length(unique(annot_overlap$Gene.refGene)))

    maf_c <- gsub("\\.", "", as.character(maf))
    maf_c <- paste0("maf", maf_c)
    command_fname <- sprintf("./extract_overlapped_exome_%s.sh", maf_c)
    
    if(file.exists(command_fname)) unlink(command_fname)
    file.create(command_fname)
    fileConn <- file(command_fname, "w")
    header_lines <- c(
        "#!/bin/bash",
        "#SBATCH --mem=60G",
        "#SBATCH --time=24:00:00",
        sprintf("#SBATCH --job-name=extract_overlapped_exome_%s", maf_c),
        sprintf("#SBATCH --output=~/project/imputation-rvtest/workflows/imputation_aggregated_analysis/scripts/extract_overlapped_exome_%s.out", maf_c),
        sprintf("#SBATCH --error=~/project/imputation-rvtest/workflows/imputation_aggregated_analysis/scripts/extract_overlapped_exome_%s.err", maf_c),
        "#SBATCH -p CSG",
        "#SBATCH --mail-type=FAIL",
        "#SBATCH --mail-user=tl3031@cumc.columbia.edu",
        "source ~/mamba_activate.sh",
        sprintf("cd ~/project/imputation-rvtest/workflows/imputation_aggregated_analysis/gene/exome_topmed_overlap/exome_%s", maf_c),
        "module load Plink/1.9.10"
    )
    writeLines(header_lines, fileConn)
    close(fileConn)

    for(g in unique(annot_overlap$Gene.refGene)){
        annot_gene <- annot_overlap %>% 
            filter(is.na(MAF_nfe_exome) | MAF_nfe_exome < maf) %>% 
            filter(Gene.refGene == g) 
        
#         annot_gene %>% 
#             select(ID_hg38) %>% 
#             fwrite(sprintf("~/project/imputation-rvtest/workflows/imputation_aggregated_analysis/gene/exome_topmed_overlap/exome_%s/%s", maf_c, g))
        
        chr <- annot_gene[1,]$Chr
        
        command <- sprintf("plink --bfile ~/project/imputation-rvtest/analysis/imputation_aggregated_analysis/exome/ukb23156_c%d_maf001_LOF_missense_extracted \\
                            --extract %s --export A --make-bed --out %s;", chr, g, g)
        fileConn <- file(command_fname, "a")
        writeLines(command, fileConn)
        close(fileConn)
    }
}

[1] 3097
[1] 3097
[1] 3097


### Writing script for job submission

In [5]:
# 1 10 1 3 0.005 "~/project/git/imputation_brv/workflow/dsc_pipeline_rsq03_maf0005"

rsq = 3

for(i in seq(1, 3203, 500)){
    for(j in c(2,11)){
        for(maf in c(0.01, 0.005, 0.001)){
            
            out_dir = "/home/tl3031/project/git/imputation_brv/workflow/exome_topmed_overlap"

            start = i
            end = start + 500 - 1
            
            if(end > 3203) end = 3203

            command_script_fname <- sprintf("~/project/git/imputation_brv/workflow/scripts/run_overlap_rsq0%i_maf%.3f_bin_phenotype%i_%i_%i.sh", rsq, maf, j, start, end)
            if(file.exists(command_script_fname)) unlink(command_script_fname)

            file.create(command_script_fname)
            fileConn <- file(command_script_fname, "w")
            header_lines <- c(
                "#!/bin/bash",
                "#SBATCH --mem=60G",
                "#SBATCH --time=24:00:00",
                sprintf("#SBATCH --job-name=run_overlap_rsq0%i_maf%.3f_bin_phenotype%i_%i_%i", rsq, maf, j, start, end),
                sprintf("#SBATCH --output=/mnt/vast/hpc/csg/tl3031/git/imputation_brv/workflow/scripts/run_overlap_rsq0%i_maf%.3f_bin_phenotype%i_%i_%i.out", rsq, maf, j, start, end),
                sprintf("#SBATCH --error=/mnt/vast/hpc/csg/tl3031/git/imputation_brv/workflow/scripts/run_overlap_rsq0%i_maf%.3f_bin_phenotype%i_%i_%i.err", rsq, maf, j, start, end),
                "#SBATCH -p CSG",
                "#SBATCH --mail-type=FAIL",
                "#SBATCH --mail-user=tl3031@cumc.columbia.edu",
                "source ~/mamba_activate.sh",
                "cd /home/tl3031/project/git/imputation_brv/workflow/scripts",
                sprintf("Rscript exome_topmed_overlap.R %i %i %i %i %.3f %s", start, end, j, rsq, maf, out_dir)
            )
            writeLines(header_lines, fileConn)
            close(fileConn)
        }
    }
}

### Summarizing result

In [8]:
exome_topmed_dir <- "~/project/git/imputation_brv/workflow/exome_topmed_overlap"
exome_topmed_flst <- list.files(exome_topmed_dir, pattern = ".csv", full.names = TRUE)

exome_topmed_result <- data.frame(matrix(nrow=0, ncol=8))
colnames(exome_topmed_result) <- c("gene", "sample", "es", "causal_prop", "prev", "zstat", "pval", "maf")

for(f in exome_topmed_flst){
    gene_res <- fread(f)
    if(ncol(gene_res) == 8) exome_topmed_result <- rbind(exome_topmed_result, gene_res)
}

exome_topmed_result <- exome_topmed_result %>% 
    mutate(gene = sapply(stringr::str_split(gene, "_"), function(x) x[2]))

In [9]:
exome_topmed_result %>%
    group_by(sample, prev, maf) %>% 
    filter(pval < 2.5e-6) %>% 
    summarise(power = n()/3203) %>%
    arrange(prev, desc(maf)) %>%
    select(prev, maf, sample, power)

`summarise()` has grouped output by 'sample', 'prev'. You can override using
the `.groups` argument.


prev,maf,sample,power
<dbl>,<dbl>,<chr>,<dbl>
0.1,0.010,exome,0.5881986
0.1,0.010,topmed,0.5316890
0.1,0.005,exome,0.5316890
0.1,0.005,topmed,0.4564471
0.1,0.001,exome,0.3290665
0.1,0.001,topmed,0.2116766
0.2,0.010,exome,0.5816422
0.2,0.010,topmed,0.5313768
0.2,0.005,exome,0.5238839


In [13]:
exome_topmed_result %>% filter(sample == "topmed") %>% filter(pval < 2.5e-6) %>% group_by(prev, maf) %>% summarise(power = n()/3203)

`summarise()` has grouped output by 'prev'. You can override using the
`.groups` argument.


prev,maf,power
<dbl>,<dbl>,<dbl>
0.1,0.001,0.2116766
0.1,0.005,0.4564471
0.1,0.010,0.5316890
0.2,0.001,0.2166719
0.2,0.005,0.4598814
0.2,0.010,0.5313768


In [10]:
for(maf in c(0.01, 0.005, 0.001)){
    annot_overlap <- annot %>% filter(MAF_nfe_exome < maf, !is.na(R2_topmed), R2_topmed != 0, R2_exome == 999)
    annot_overlap %>% group_by(Gene.refGene) %>% summarise(mean_r2_gene = mean(R2_topmed)) %>% pull(mean_r2_gene) %>% mean() %>% print()
}

[1] 0.6944453
[1] 0.6925143
[1] 0.6857904


## Average R2 for all genes and significant genes

We are interested in whether the average R2 will be different for all genes vs the genes reaching exome wide significant level.

In [28]:
library(dplyr)
library(data.table)

setwd("~/project/git/imputation_brv/workflow/results")

In [29]:
result_rsq03_maf001 <- fread("rsq03_maf001_result.csv") %>% filter(sample %in% c("topmed", "hrc"), es == 1.5, causal_prop == 1)
result_rsq03_maf0005 <- fread("rsq03_maf0005_result.csv") %>% filter(sample %in% c("topmed", "hrc"), es == 1.5, causal_prop == 1)
result_rsq03_maf0001 <- fread("rsq03_maf0001_result.csv") %>% filter(sample %in% c("topmed", "hrc"), es == 1.5, causal_prop == 1)

In [65]:
hrc_chr1 <- fread("~/project/imputation-rvtest/analysis/imputation_aggregated_analysis/hrc_topmed/hrc_168206ids_chr1_rsq03_maf001_annot.csv.gz")
hrc_chr2 <- fread("~/project/imputation-rvtest/analysis/imputation_aggregated_analysis/hrc_topmed/hrc_168206ids_chr2_rsq03_maf001_annot.csv.gz")

topmed_chr1 <- fread("~/project/imputation-rvtest/analysis/imputation_aggregated_analysis/hrc_topmed/topmed_v3_168206ids_chr1_rsq03_maf001_annot.csv.gz")
topmed_chr2 <- fread("~/project/imputation-rvtest/analysis/imputation_aggregated_analysis/hrc_topmed/topmed_v3_168206ids_chr2_rsq03_maf001_annot.csv.gz")

hrc_annot <- rbind(hrc_chr1, hrc_chr2)
topmed_annot <- rbind(topmed_chr1, topmed_chr2)

In [77]:
result_df <- data.frame(matrix(ncol = 4, nrow = 0))
colnames(result_df) <- c("dataset", "prev", "maf", "r2")

for(p in c(0.1, 0.2)){
    for(m in c(0.01, 0.005, 0.001)){
        sig_gene_lst_topmed <- all_gene %>% filter(sample == "topmed", prev == p, maf == m, pval < 2.5e-6) %>% pull(gene)
        sig_gene_lst_hrc <- all_gene %>% filter(sample == "hrc", prev == p, maf == m, pval < 2.5e-6) %>% pull(gene)

        mean_r2_all_gene_topmed <- topmed_annot %>% 
            filter(MAF_nfe_exome < m) %>% 
            group_by(Gene.refGene) %>% summarise(mean_r2 = mean(R2)) %>%
            pull(mean_r2) %>% mean()
        
        mean_r2_all_gene_hrc <- hrc_annot %>% 
            filter(MAF_nfe_exome < m) %>% 
            group_by(Gene.refGene) %>% summarise(mean_r2 = mean(R2)) %>%
            pull(mean_r2) %>% mean()
        
        mean_r2_sig_gene_topmed <- topmed_annot %>% 
            filter(MAF_nfe_exome < m, Gene.refGene %in% sig_gene_lst_topmed) %>% 
            group_by(Gene.refGene) %>% summarise(mean_r2 = mean(R2)) %>%
            pull(mean_r2) %>% mean()
        
        mean_r2_sig_gene_hrc <- hrc_annot %>% 
            filter(MAF_nfe_exome < m, Gene.refGene %in% sig_gene_lst_hrc) %>% 
            group_by(Gene.refGene) %>% summarise(mean_r2 = mean(R2)) %>%
            pull(mean_r2) %>% mean()
        
        sub_df <- data.frame(dataset = c("hrc", "topmed"), prev = c(p, p), maf = c(m, m),
                             r2 = c(sprintf("%.3f (%.3f)", mean_r2_all_gene_hrc, mean_r2_sig_gene_hrc), 
                                    sprintf("%.3f (%.3f)", mean_r2_all_gene_topmed, mean_r2_sig_gene_topmed)))
        result_df <- rbind(result_df, sub_df)
    }

}
result_df %>% arrange(dataset, prev, desc(maf))

dataset,prev,maf,r2
<chr>,<dbl>,<dbl>,<chr>
hrc,0.1,0.010,0.723 (0.744)
hrc,0.1,0.005,0.714 (0.734)
hrc,0.1,0.001,0.682 (0.707)
hrc,0.2,0.010,0.723 (0.743)
hrc,0.2,0.005,0.714 (0.734)
hrc,0.2,0.001,0.682 (0.707)
topmed,0.1,0.010,0.677 (0.682)
topmed,0.1,0.005,0.675 (0.680)
topmed,0.1,0.001,0.669 (0.673)
